# Toxic Comment Classification Challenge

## https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

In [1]:
import numpy as np
import pandas as pd

from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.preprocessing import text, sequence

from collections import Counter

Using TensorFlow backend.


### Hyperparameters

In [2]:
hyper_params = {
    'validation_split': 0.1,
    'batch_size': 32,
    'epochs': 10,
    'embedding_size': 128,
    'keep_probability': 0.9,
    'lstm_size': 50,
    'dense_size': 50,
    'max_sequence': 100
}

### Prepare dataset

In [3]:
# Download from https://www.kaggle.com/c/8076/download/train.csv.zip
train = pd.read_csv('data/train.csv')
train_sentences = train.comment_text.values

# Download from https://www.kaggle.com/c/8076/download/test.csv.zip
test = pd.read_csv('data/test.csv')
test_sentences = test.comment_text.values

CLASSES = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(np.concatenate((train_sentences, test_sentences), axis=0))
train_tokenized = tokenizer.texts_to_sequences(train_sentences)
test_tokenized = tokenizer.texts_to_sequences(test_sentences)
vocab_size = len(Counter([token for sublist in train_tokenized + test_tokenized for token in sublist])) + 1

X_train = sequence.pad_sequences(train_tokenized, maxlen=hyper_params['max_sequence'])
y_train = train[CLASSES].values
X_test = sequence.pad_sequences(test_tokenized, maxlen=hyper_params['max_sequence'])

### Build the network

In [4]:
model = Sequential()

model.add(Embedding(vocab_size, hyper_params['embedding_size']))
model.add(LSTM(hyper_params['lstm_size']))
model.add(Dropout(1 - hyper_params['keep_probability']))
model.add(Dense(hyper_params['dense_size'], activation='relu'))
model.add(Dropout(1 - hyper_params['keep_probability']))
model.add(Dense(len(CLASSES), activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

### Train the network

In [5]:
history = model.fit(X_train, y_train, batch_size=hyper_params['batch_size'], epochs=hyper_params['epochs'], 
                    validation_split=hyper_params['validation_split'])

Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143613/143613 [==============================] - 3091s 22ms/step - loss: 0.0650 - acc: 0.9788 - val_loss: 0.0509 - val_acc: 0.9815
Epoch 2/10
143613/143613 [==============================] - 3217s 22ms/step - loss: 0.0413 - acc: 0.9840 - val_loss: 0.0507 - val_acc: 0.9818
Epoch 3/10
143613/143613 [==============================] - 2974s 21ms/step - loss: 0.0323 - acc: 0.9872 - val_loss: 0.0514 - val_acc: 0.9815
Epoch 4/10
143613/143613 [==============================] - 3396s 24ms/step - loss: 0.0252 - acc: 0.9901 - val_loss: 0.0585 - val_acc: 0.9790
Epoch 5/10
143613/143613 [==============================] - 3323s 23ms/step - loss: 0.0191 - acc: 0.9925 - val_loss: 0.0712 - val_acc: 0.9778
Epoch 6/10
143613/143613 [==============================] - 3329s 23ms/step - loss: 0.0143 - acc: 0.9944 - val_loss: 0.0848 - val_acc: 0.9778
Epoch 7/10
143613/143613 [==============================] - 3349s 23ms/step - loss: 0.0110 - acc:

### Prepare submission file

In [6]:
# Download from https://www.kaggle.com/c/8076/download/sample_submission.csv.zip
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission[CLASSES] = model.predict(X_test)
sample_submission.to_csv('submission.csv', index=False)